In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService
from src.utils.type_utils import create_langchain_doc

DATA_REPO = "BroDeadlines/TEST.basic_test_tdt_dataset"
SPLIT = "INDEX.medium_index_TDT"
provider = ProviderService()

# Push Hugging Face dataset to ES

- data from hugging face
- push vector data 


In [2]:
dataset = load_dataset(DATA_REPO, split=SPLIT)
index = SPLIT.lower()

vec_idx = f"vec-{index}"
text_idx = f"text-{index}"

es = provider.get_elasticsearch_store(vec_idx)
bm = provider.get_elasticsearch_bm25(text_idx)

In [3]:
dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits'],
    num_rows: 344
})

In [4]:
from time import time
start = time()
a = [212]*455
end = time()
print(end - start)

4.863739013671875e-05


In [5]:
start = time()
for row in dataset:
    meta = {"doc_id": row['doc_id']}
    docs = [create_langchain_doc(content=split, metadata=meta) for split in row['splits']]
    es.add_documents(docs)
    bm.add_documents(docs)

end = time()

In [6]:
(end-start) / 60

5.594338897864024

In [8]:
vec_idx

'vec-index.medium_index_tdt'

In [9]:
text_idx

'text-index.medium_index_tdt'

# Push Proposition dataset to ES

In [2]:
DATA_REPO = "BroDeadlines/TEST.edu_tdt_proposition_data"
# SPLIT = "propositon_medium_edu_tdt"
SPLIT = "INDEX.medium_index_TDT_clean"

proposition_ds = load_dataset(DATA_REPO, split=SPLIT)

Generating propositon_medium_edu_tdt split:   0%|          | 0/344 [00:00<?, ? examples/s]

Generating INDEX.medium_index_TDT split:   0%|          | 0/344 [00:00<?, ? examples/s]

Generating INDEX.medium_index_TDT_clean split:   0%|          | 0/344 [00:00<?, ? examples/s]

In [4]:
proposition_ds[0]['propositions']

['Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến.\nSinh viên nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến.\nSinh viên chọn học kỳ muốn gia hạn học phí.\nSinh viên chọn loại đơn cần nộp.\nSinh viên tiến hành điền các thông tin cần thiết.\nSinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không.\nViệc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí.\nSinh viên đính kèm minh chứng khi nộp đơn gia hạn thời gian đóng học phí.\nSinh viên nộp đơn gia hạn thời gian đóng học phí về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.']

In [5]:
from collections import Counter
lengths = []
list_lengths = []

def check_length(row):
    lnstr = f"len{len(row['propositions'])}"
    list_lengths.append(f"len{len(row['proposition_list'])}")
    lengths.append(lnstr)
    return

proposition_ds.map(check_length)
print(Counter(lengths))
# Counter(list_lengths)

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

Counter({'len1': 344})


In [6]:
proposition_filtered_ds = proposition_ds.filter(lambda e: e['url'] == "https://tuvanhocduong.tdtu.edu.vn/News" and sum([len(a) for a in e['propositions']]) > 0)
proposition_filtered_ds

Filter:   0%|          | 0/344 [00:00<?, ? examples/s]

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 144
})

In [7]:
import json
a = proposition_ds[0]['propositions']
b = json.loads(a[0])
c = "\n".join(b)
print(c)

Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến.
Sinh viên nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến.
Sinh viên chọn học kỳ muốn gia hạn học phí.
Sinh viên chọn loại đơn cần nộp.
Sinh viên tiến hành điền các thông tin cần thiết.
Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không.
Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.
Nếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí.
Sinh viên đính kèm minh chứng khi nộp đơn gia hạn thời gian đóng học phí.
Sinh viên nộp đơn gia hạn thời gian đóng học phí về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.


In [28]:
len(errors)

268151

## Full-text option

In [7]:
index = SPLIT.lower()

vec_idx = f"vec-sentence-{index}"
text_idx = f"text-sentence-{index}"

es = provider.get_elasticsearch_store(vec_idx)
bm = provider.get_elasticsearch_bm25(text_idx)

errors = []

def build_doc_upload(row):
    # full-text
    propos = row['propositions']
    meta = {"doc_id": row['doc_id']}
    docs = [create_langchain_doc(content=prop, metadata=meta) for prop in row['propositions']]
    try:
        es.add_documents(docs)
        bm.add_documents(docs)
    except:
        errors.append(meta['doc_id'])
        print("ERROR")
    return

def build_doc_upload_v2(row):
    # sentence-level
    propos = row['propositions']
    meta = {"doc_id": row['doc_id']}
    docs = [create_langchain_doc(content=prop, metadata=meta) for prop in row['proposition_list']]
    try:
        es.add_documents(docs)
        bm.add_documents(docs)
    except:
        errors.append(meta['doc_id'])
        print("ERROR")
    return

In [8]:
print(vec_idx)
text_idx

vec-sentence-index.medium_index_tdt_clean


'text-sentence-index.medium_index_tdt_clean'

In [9]:
proposition_ds.map(build_doc_upload)

Parameter 'function'=<function build_doc_upload at 0x7f3da82d2820> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/344 [00:00<?, ? examples/s]

ERROR


Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 344
})

In [10]:
errors

['ebe87ce2-13cc-11ef-b548-0242ac1c000c']